In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train/255
x_test = x_test/255
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
from spkeras.models import cnn_to_snn
from spkeras.utils import save_pickle, load_pickle

from tensorflow.keras.models import load_model

with tf.device(gpu):
    cnn_mdl = load_model('../../cnn_model/cnn_mdl.h5')

In [ ]:
#evalute thresholding over different timesteps
timesteps = [16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256]

thresholding = [0,0.5]
scaling_factor = [1]

cnn_result = []
snn_result = []

_, cnn_acc = cnn_mdl.evaluate(x_test,y_test)
snn_mdl = cnn_to_snn()(cnn_mdl,x_train)

result = []
for s1 in thresholding:
    _result = []
    for s2 in scaling_factor:
        __result = []
        for t in timesteps:
            _,acc = snn_mdl.evaluate(x_test,y_test,timesteps=t,
                                        thresholding=s1,                                        
                                        scaling_factor=s2)
            __result.append(acc)
        _result.append(__result)
        
    result.append(_result)
    
cnn_result.append(cnn_acc)
snn_result.append(result)   
             
    
save_pickle(snn_result,'mdl'+'_snn_result','../results/')
save_pickle(cnn_result,'mdl'+'_cnn_result','../results/')

In [ ]:
#evalute bitwidth over different timesteps
timesteps = [16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256]
thresholding = [0.5]
scaling_factor = [1]
bit = [7,8,9,10]

cnn_result = []
snn_result = []

_, cnn_acc = cnn_mdl.evaluate(x_test,y_test)
cnn_result.append(cnn_acc)
result_=[]
for b in bit:
    snn_mdl = cnn_to_snn(signed_bit=b)(cnn_mdl,x_train)
    
    result = []
    for s1 in thresholding:
        _result = []
        for s2 in scaling_factor:
            __result = []
            for t in timesteps:
                _,acc = snn_mdl.evaluate(x_test,y_test,timesteps=t,
                                            thresholding=s1,                                        
                                            scaling_factor=s2,
                                            spike_ext=0)
                __result.append(acc)
            _result.append(__result)
            
        result.append(_result)
    result_.append(result)
        
snn_result.append(result_)   
             
    
save_pickle(snn_result,'mdl_bit'+'_snn_result','../results/')
save_pickle(cnn_result,'mdl_bit'+'_cnn_result','../results/')

In [ ]:
#extract MOps over different timesteps
timesteps = [16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256]

thresholding = [0,0.5]
scaling_factor = [1]

import numpy as np

    
cnn_result = []
snn_result = []
snn_result_ = []

_, cnn_acc = cnn_mdl.evaluate(x_test,y_test)
snn_mdl = cnn_to_snn()(cnn_mdl,x_train)

result = []
result_ = []
shape,Neuros = snn_mdl.NeuronNumbers(mode=1) #extract connections

for s1 in thresholding:
    _result = []
    _result_ = []
    for s2 in scaling_factor:
        __result = []
        __result_ = []
        for t in timesteps:
            #extract spikes
            l,N = snn_mdl.SpikeCounter(x_train,timesteps=t,
                                        thresholding=s1,                                        
                                        scaling_factor=s2)
            #Count operations 
            inp_spikes = x_train*t
            inp_spikes = np.floor(inp_spikes)
            inp_spikes = (np.sum(inp_spikes)/np.prod(x_train.shape))*np.prod(shape[0])
            inp_spikes = np.ceil(inp_spikes)
            
            a = []
            for k in range(len(Neuros)):
                if k == len(Neuros) - 1:
                    continue
                _a = N[k]/Neuros[k]*np.prod(shape[k+1])
                a.append(_a)
                
            __result_.append(np.max(l))
            __result.append(np.floor(np.sum(a+inp_spikes)))
        _result.append(__result)
        _result_.append(__result_)
    result.append(_result)
    result_.append(_result_)
    
snn_result.append(result) 
snn_result_.append(result)
             
save_pickle(snn_result,'mdl_spikes'+'_snn_result','../results/')
save_pickle(snn_result_,'mdl_max_train'+'_snn_result','../results/')        

In [ ]:
#Count neurons under different spike numbers at 64 timesteps (None loss mode)
timesteps = [64]
thresholding = [0]
scaling_factor = [1]

cnn_result = []
snn_result = []

_, cnn_acc = cnn_mdl.evaluate(x_test,y_test)
snn_mdl = cnn_to_snn()(cnn_mdl,x_train)

result = []
for s1 in thresholding:
    _result = []
    for s2 in scaling_factor:
        __result = []
        for t in timesteps:
            _,N = snn_mdl.SpikeCounter(x_train,timesteps=t,
                                       thresholding=s1,                                        
                                       scaling_factor=s2,
                                       noneloss=True,
                                       mode=1)
            
            __result.append(N)
            
        _result.append(__result)
        
    result.append(_result)

snn_result.append(result) 
             
save_pickle(snn_result,'mdl_spikes_count64_noneloss'+'_snn_result','../results/')    